# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>1. Initial Setting</b></div>

#### <b><span style='color:#FFA500'> | </span> Import Libraries</b>

In [ ]:
import os
import gc 
import ctypes
import random
import time
import re
import copy
from tqdm.auto import tqdm, trange
from sklearn.decomposition import PCA

import pandas as pd, numpy as np
import polars as pl # For Feature Engineering

import matplotlib.pyplot as plt
import seaborn as sns 

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' # For GPU T4x2

import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score

import warnings 
warnings.filterwarnings('ignore')

#### <b><span style='color:#FFA500'> | </span> Configure</b>

In [ ]:
class CFG:
    SEED = 2024
    VER = 4
    LOAD_MODELS_FROM = None
    BASE_PATH = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'

#### <b><span style='color:#FFA500'> | </span> Clean Memory</b>

In [ ]:
Clean = True

def clean_memory():
    if Clean:
        ctypes.CDLL('libc.so.6').malloc_trim(0)
        gc.collect()
        
clean_memory()        

#### <b><span style='color:#FFA500'> | </span> Seed Everything</b>

In [ ]:
def seed_everything(): # To proudce simliar result in each run 
    random.seed(CFG.SEED)
    np.random.seed(CFG.SEED)
    os.environ['PYTHONHASHSEED'] = str(CFG.SEED)
    
seed_everything()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>2. Road and Read Data</b></div>

#### <b><span style='color:#FFA500'> | </span> Meta Data</b>


- **`[train/test].csv`**
  - `essay_id`: The unique ID of the essay
  - `full_text`: The full essay reponse
  - `score`: Holistic score of the essay on a 1-6 scale
  
  
- **`sample_submission.csv`**
  - `essay_id`: The unique ID of the essay
  - `score`: The predicted holistic score of the essay on a 1-6 scale
  
  
> [Link to the Holistic Scoring Rubric](https://storage.googleapis.com/kaggle-forum-message-attachments/2733927/20538/Rubric_%20Holistic%20Essay%20Scoring.pdf)

#### <b><span style='color:#FFA500'> | </span> Train Data</b>

In [ ]:
df_train = pd.read_csv(CFG.BASE_PATH + 'train.csv')

print('Shape of Train: ', df_train.shape)
print(display(df_train.head()))

In [ ]:
df_train.info()

In [ ]:
categorical_columns = df_train.select_dtypes(include=['object','category']).columns
categorical_summary = df_train[categorical_columns].describe()
categorical_summary

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x=df_train['score'])
plt.title('Distribution of Score')
plt.xlabel('Score of Essay')
plt.ylabel('Frequency')
plt.show()

As you know, It seems like Score of Essay is imbalance. So I'm gonna use Cross Validation(Stratified CV)

#### <b><span style='color:#FFA500'> | </span> Test Data</b>

In [ ]:
df_test = pd.read_csv(CFG.BASE_PATH + 'test.csv')

print('Shape of Test: ', df_test.shape)
print(display(df_test.head()))

In [ ]:
df_train.iloc[0].full_text

📌 **Check Out**

- 3 Test set is just placeholder
- Public Test set is 30% of the test data
- Private Test set is other 70% of the test data

- The Private Score is based on other 70% of the test data, So it is different with public score 

#### <b><span style='color:#FFA500'> | </span> Pandas -> Polars</b>

📌 **Why Polars??**
- Unlike DL, in ML, we have to make features
- But, There are 17307 sample. So it will take times a lot 
- Polras is much effective than pandas when extracting features from data
- Before Learning ML, we're gonna return to Pandas DataFrame(ex: df.to_pandas)


In [ ]:
# os.path.join(CFG.BASE_PATH,'train.csv')
# CFG.BASE_PATH + 'train.csv'

train = pl.from_pandas(df_train).with_columns(
    pl.col('full_text').str.split(by='\n\n').alias('paragraph'))
test = pl.read_csv(os.path.join(CFG.BASE_PATH,'test.csv')).with_columns(
    pl.col('full_text').str.split(by='\n\n').alias('paragraph')
)

schema_train = train.schema # MetaData
schema_test = test.schema # MetaData

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>3. Feature Engineering</b></div>

#### <b><span style='color:#FFA500'> | </span> Preprocessing</b>

In [ ]:
def removeHTML(x):
    html = re.compile(r'<.*?>')
    return html.sub(r'',x) # html -> ''

def dataPreprocessing(x):
    
    x = x.lower()
    # remove html
    x = removeHTML(x)
    
    #remove string with starting @
    x = re.sub("@\w+",'',x)
    
    # remove number
    x = re.sub("'\d+",'',x)
    x = re.sub("\d+",'',x)
    
    # remove url
    x = re.sub("http\w+",'',x)
    
    # Replace consecutive empty spaces with a single empty spaces
    x = re.sub(r'\s+', " ", x)
    
    # Replace consecutive commas and periods with a single comma and period
    x = re.sub(r'\.+', ".",x)
    x = re.sub(r'\,+', ".",x)
    # Remove empty character at the beginning and end
    x = x.strip()
    return x

#### <b><span style='color:#FFA500'> | </span> Paragraph Features</b>

In [ ]:
paragraph_features = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']

def Paragraph_Features(x):
    # Expand the paragraph list to several lines of data
    x = x.explode('paragraph') 
    
    print('Paragraph Preprocessing')
    x = x.with_columns(
         pl.col('paragraph').map_elements(dataPreprocessing)
    )
    
    print('Caculate the length of each paragraph')
    x = x.with_columns(
         pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len")
    )
    
    print('Caculate the number of sentences and words in each paragraph')
    x = x.with_columns(
         pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias('paragraph_sentence_cnt'),
         pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias('paragraph_word_cnt')
    )
    return x

In [ ]:
%%time

train_feats = Paragraph_Features(train)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats.to_pandas()
sns.pairplot(data=tmp.drop(columns=['essay_id','full_text','paragraph']), hue='score', palette='bright')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
sns.boxplot(y=tmp['paragraph_len'], x=tmp['score'])
plt.title("Distribution of paragraph_len")
plt.xlabel('paragraph_len')
plt.ylabel('Frequency')

plt.subplot(1,3,2)
sns.boxplot(y=tmp['paragraph_sentence_cnt'], x=tmp['score'])
plt.title("Distribution of paragraph_sentence_cnt")
plt.xlabel('paragraph_sentence_cnt')
plt.ylabel('Frequency')


plt.subplot(1,3,3)
sns.boxplot(y=tmp['paragraph_word_cnt'], x=tmp['score'])
plt.title("Distribution of paragraph_word_cnt")
plt.xlabel('paragraph_word_cnt')
plt.ylabel('Frequency')


plt.show()

In [ ]:
def Paragraph_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f'paragraph_{i}_cnt') for i in [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]],
    
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in paragraph_features],
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in paragraph_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in paragraph_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in paragraph_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in paragraph_features],
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    
    df = df.to_pandas() # polars -> pandas
    
    return df

In [ ]:
%%time

train_feats = Paragraph_aggregation(train_feats)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
for i, col in enumerate(tmp.columns):
    plt.subplot(3,3,i+1)
    for idx in range(1,7):
        sns.kdeplot(x=tmp[tmp['score']==idx][col], label=idx)
        plt.legend()
    if i == 8: break
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Correlation of Paragraph Feature with score')
tmp = train_feats.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
sns.heatmap(tmp.corr(), annot=True, cmap='coolwarm', fmt='.1f')
plt.show()

#### <b><span style='color:#FFA500'> | </span> Sentence Features</b>

In [ ]:
sentence_features = ['sentence_len','sentence_word_cnt']

def Sentence_Features(x):
    print('Preprocess full_text and use periods to segment sentences in the text')
    x = x.with_columns(
        pl.col('full_text').map_elements(lambda x: dataPreprocessing(x)).str.split(".").alias('sentence')
    )
    x = x.explode('sentence')
    
    print('Caculate the length of a sentence') 
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: len(x)).alias('sentence_len'))
    
    print('Count the number of words in each sentence')
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: len(x.split(" "))).alias("sentence_word_cnt"))

    return x

In [ ]:
%%time 

train_feats2 = Sentence_Features(train)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats2.to_pandas()
sns.pairplot(data=tmp.drop(columns=['essay_id','full_text','sentence','paragraph']), hue='score', palette='bright')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.boxplot(y=tmp['sentence_len'], x=tmp['score'])
plt.title("Distribution of sentence_len")
plt.xlabel('sentence_len')
plt.ylabel('Frequency')

plt.subplot(1,2,2)
sns.boxplot(y=tmp['sentence_word_cnt'], x=tmp['score'])
plt.title("Distribution of sentence_word_cnt")
plt.xlabel('sentence_word_cnt')
plt.ylabel('Frequency')

plt.show()

In [ ]:
def Sentence_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f'sentence_{i}_cnt') for i in [5,10,15,25,30,40,50,60,70,80,90,100]],
    
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in sentence_features],
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in sentence_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in sentence_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in sentence_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in sentence_features],   
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    
    df = df.to_pandas() # polars -> pandas
    
    return df

In [ ]:
train_feats2 = Sentence_aggregation(train_feats2)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats2.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
for i, col in enumerate(tmp.columns):
    plt.subplot(3,3,i+1)
    for idx in range(1,7):
        sns.kdeplot(x=tmp[tmp['score']==idx][col], label=idx)
        plt.legend()
    if i == 8: break
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Correlation of Sentence Feature with score')
tmp = train_feats2.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
sns.heatmap(tmp.corr(), annot=True, cmap='coolwarm', fmt='.1f')
plt.show()

#### <b><span style='color:#FFA500'> | </span> Word Features</b>

In [ ]:
word_features = ['word_len',]

def Word_Features(x):
    print('Preprocess full_text and use spaces to seperate words fro the text')
    x = x.with_columns(
        pl.col('full_text').map_elements(lambda x: dataPreprocessing(x)).str.split(" ").alias('word')
    )
    x = x.explode('word')
    
    print('Caculate the length of a word') 
    x = x.with_columns(
        pl.col('word').map_elements(lambda x: len(x)).alias('word_len'))
    

    return x

In [ ]:
%%time

train_feats3 = Word_Features(train)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats3.to_pandas()
sns.pairplot(data=tmp.drop(columns=['essay_id','full_text','sentence','paragraph','word']), hue='score', palette='bright')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y=tmp['word_len'], x=tmp['score'])
plt.title("Distribution of word_len")
plt.xlabel('word_len')
plt.ylabel('Frequency')

plt.show()

In [ ]:
def Word_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('word').filter(pl.col('word_len') >= i).count().alias(f'sentence_{i}_cnt') for i in [2.0,2.5,3.0,3.5,4.0,4.5,5.0,6.0]],
    
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in word_features],
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in word_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in word_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in word_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in word_features],   
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    
    df = df.to_pandas() # polars -> pandas
    
    return df

In [ ]:
train_feats3 = Word_aggregation(train_feats3)

In [ ]:
plt.figure(figsize=(12,6))
tmp = train_feats3.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
for i, col in enumerate(tmp.columns):
    plt.subplot(3,3,i+1)
    for idx in range(1,7):
        sns.kdeplot(x=tmp[tmp['score']==idx][col], label=idx)
        plt.legend()
    if i == 8: break
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Correlation of Word Feature with score')
tmp = train_feats3.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')
sns.heatmap(tmp.corr(), annot=True, cmap='coolwarm', fmt='.1f')
plt.show()

#### <b><span style='color:#FFA500'> | </span> Tf-idf features</b>

In [ ]:
tmp = train.with_columns(pl.col('full_text').map_elements(lambda x: dataPreprocessing(x)))

In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
    tokenizer = lambda x: x,
    preprocessor = lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer= 'word',
    ngram_range = (1,3),
    min_df =0.05,
    max_df=0.95,
    sublinear_tf = True # Term Frequency Log Scaling 
)


# Fit all datasets into TfidfVectorizer
train_tfid = vectorizer.fit_transform([i for i in tmp['full_text']])

              
# Convert to array
dense_matrix = train_tfid.toarray()

# Convert to dataframe
df = pd.DataFrame(dense_matrix)

In [ ]:
df['essay_id'] = df_train['essay_id'].values

plt.figure(figsize=(12,6))
tmp = df.merge(df_train.drop(columns='full_text'), on='essay_id',  how='left') 
tmp = tmp.drop(columns='essay_id')

for i in range(1,7):
    plt.subplot(2,3,i)
    sns.boxplot(x=tmp['score'] ,y = tmp[i])

plt.show()

In [ ]:
plt.figure(figsize=(14,10))

plt.title("Histogram of TfidfVectorizer")
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.hist(x=df[i], bins=30, color='red', edgecolor='black')
    
plt.show()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>4. Save train.csv</b></div>

In [ ]:
train = train_feats.merge(train_feats2, on='essay_id', how='left')
train = train.merge(train_feats3, on='essay_id', how='left')
train = train.merge(df, on='essay_id', how='left')

In [ ]:
train.to_csv('train.csv', index=None)
display(train.head())